In [1]:
!pip install shap
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import mutual_info_classif
import lightgbm as lgb
import pandas as pd
import numpy as np
import seaborn as sns
import shap
import matplotlib.pyplot as plt

     |████████████████████████████████| 356 kB 7.8 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491652 sha256=ca71bd7270d25febc2c7f65af9fff6881e328a9c6cdbd3f849b48e7d444114bc
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [2]:
!nvidia-smi
!cat /etc/issue

Fri Sep 17 06:05:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
from google.colab import drive
drive.mount('/content/drive')

cuda:0
Mounted at /content/drive


In [4]:
X = np.load('/content/drive/My Drive/ember_x_medium.npy')
y = np.load('/content/drive/My Drive/ember_y_medium.npy')

In [5]:
# class 
class DataSet:  # A class for using DataLoader
    def __init__(self, X_train, y_train):
        self.x = X_train
        self.y = y_train

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

class Encoder(torch.nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, 1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  
        self.dropout(x)
        return x

class Decoder(torch.nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.fc1 = torch.nn.Linear(512, 1024)
        self.fc2 = torch.nn.Linear(1024, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.tanh(x)  
        return x


class AutoEncoder(torch.nn.Module):
    def __init__(self, org_size):
        super().__init__()
        self.enc = Encoder(org_size)
        self.dec = Decoder(org_size)

    def forward(self, x):
        x = self.enc(x)  # encode
        x = self.dec(x)  # decode
        return x

    def encode(self, x):
        return self.enc(x)

    def decode(self, x):
        return self.dec(x)

class LightNN(nn.Module):
    def __init__(self, input_features):
        super(LightNN, self).__init__()
        n_features = 2351
        depth = 2
        layer = []
        bn = []
        layer.append(nn.Linear(input_features, n_features, bias=False))
        bn.append(nn.BatchNorm1d(n_features))
        for i in range(depth):
            layer.append(nn.Linear(n_features, n_features, bias=False))
            bn.append(nn.BatchNorm1d(n_features))
        self.fc = nn.Linear(n_features, 1, bias=False)
        self.layer = nn.ModuleList(layer)
        self.bn = nn.ModuleList(bn)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        for i in range(len(self.layer)):
            x = self.layer[i](x)
            x = nn.functional.relu(x)
            x = self.bn[i](x)
            x = nn.functional.relu(x)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc(x))
        return x

class reducedNN(nn.Module):
    def __init__(self, input_features):
        super(reducedNN, self).__init__()
        n_features = 2351
        depth = 2
        layer = []
        bn = []
        layer.append(nn.Linear(input_features, n_features, bias=False))
        bn.append(nn.BatchNorm1d(n_features))
        for i in range(depth):
            layer.append(nn.Linear(n_features, n_features, bias=False))
            bn.append(nn.BatchNorm1d(n_features))
        self.fc = nn.Linear(n_features, 1, bias=False)
        self.layer = nn.ModuleList(layer)
        self.bn = nn.ModuleList(bn)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        for i in range(len(self.layer)):
            x = self.layer[i](x)
            x = nn.functional.relu(x)
            x = self.bn[i](x)
            x = nn.functional.relu(x)
            # x = self.dropout(x)
        x = self.dropout(x)
        x = torch.sigmoid(self.fc(x))
        return x


In [6]:
criterion = nn.MSELoss()
def test(net, xtest, ytest):
    outputs = net(xtest)
    ytest = torch.reshape(ytest, (ytest.shape[0], -1))
    test_loss = criterion(outputs, ytest)
    predicted = torch.round(outputs.data)
    np_predicted = predicted.cpu().numpy().astype(np.int64)
    np_ytest = ytest.cpu().numpy()
    each_pred = np.zeros([2, 2])
    # TN:00 FP:01 TP:11 FN:10
    for i, v in enumerate(np_ytest):
        if v == 0:  # true: 0, pred:0 or 1
            each_pred[0][np_predicted[i]] += 1
        else:  # true: 1, pred:0 or 1
            each_pred[1][np_predicted[i]] += 1
    acc =  (each_pred[0][0] + each_pred[1][1])/(each_pred[0][0] + each_pred[1][1] + each_pred[0][1] + each_pred[1][0])
    fpr = each_pred[0][1]/ (each_pred[0][0] + each_pred[0][1])
    fnr = each_pred[1][0]/(each_pred[1][0]+each_pred[1][1]) # attack success rate for Validation data
    return test_loss.data, acc, fnr, fpr

def test_gbm(net, xtest, ytest):
    predicted =net.predict(xtest)
    each_pred = np.zeros([2, 2])
    predicted = predicted.astype(np.int64)
    # TN:00 FP:01 TP:11 FN:10
    for i, v in enumerate(ytest):
        if v == 0:  # true: 0, pred:0 or 1
            each_pred[0][predicted[i]] += 1
        else:  # true: 1, pred:0 or 1
            each_pred[1][predicted[i]] += 1
    acc =  (each_pred[0][0] + each_pred[1][1])/(each_pred[0][0] + each_pred[1][1] + each_pred[0][1] + each_pred[1][0])
    fpr = each_pred[0][1]/ (each_pred[0][0] + each_pred[0][1])
    fnr = each_pred[1][0]/(each_pred[1][0]+each_pred[1][1]) # attack success rate for Validation data
    return _, acc, fnr, fpr

def train(epochs, net, optim, loader, x_t, y_t):
    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        for i, (x, y) in enumerate(loader):
            y = torch.reshape(y, (y.shape[0], -1))
            loss = criterion(net(x), y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            running_loss += loss.item()
        avg_loss = running_loss / i
        if epoch % 10 == 0: # 10
            test_loss, acc, attack,fpr = test(net, x_t, y_t)
            print("Epoch:{}/{}, Avg loss:{:4.4} Test loss:{:4.4} Test accuracy:{:3.3} FPR:{:3.3}".format(epoch, epochs, avg_loss, test_loss, acc, fpr))
    net.eval() # end training (dropoutを入れている場合等に必要)

def train_ae(net, criterion, optimizer, epochs, trainloader, idx_s):
    input_dim = 2351
    bce_loss = nn.BCELoss() 
    idx_x = [i for i in range(2351) if i not in idx_s]
    h_loss = []
    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        d_running_loss = 0.0
        ae_running_loss = 0.0
        for counter, (img, y) in enumerate(trainloader, 1): # update AE for L2 loss
            optimizer.zero_grad()
            img = img.reshape(-1, input_dim)
            s = img[ : , idx_s] # extract s
            output = net(s)
            loss = criterion(output, s)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / counter
        if epoch % 10 == 0:
            print("Epoch:{}/{}, Train loss:{:3.3}".format(epoch, epochs, avg_loss))
        h_loss.append(avg_loss)
    return h_loss

In [7]:
# Parameters
L = 512 # The attacker's manipulable feature is [L : R] 
R = 2351 # The attacker's manipulable feature is [L : R]

F = []
F_defender = [i for i in range(L, R)]
N2 = len(F_defender)

lr_lightNN = 0.1  # 0.1
epochs_lightNN = 100  # 100
batch_lightNN = 512  # 512

lr_ae = 0.1 # 0.1
epochs_ae = 50 # 50
batch_ae = 512 # 

lr_reducedNN = 0.01 # 0.01
epochs_reducedNN = 200  # 300
batch_reducedNN = 512 # 128

In [10]:
netType = 2 # 1 : lightGBT 2 : nn
algType = 1 # 1: LargeAbsSHAP + MinPopulation 2: LargeSHAP + CountSHAP 3: FGSM + label flip
encOnly = False # False: use encoder + decoder True: use encoder only

epsilon = 1 # 1.5
p = 80
N = 8
nShap = 500

# split
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X, y, test_size=0.2, random_state=1) # fix random state

# normalize
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_np = scaler.fit_transform(X_train_np)
X_test_np = scaler.transform(X_test_np)

# numpy⇒tensor
X_train = torch.from_numpy(X_train_np).float().to(device=device)
y_train = torch.from_numpy(y_train_np).float().to(device=device)
X_test = torch.from_numpy(X_test_np).float().to(device=device)
y_test = torch.from_numpy(y_test_np).float().to(device=device)

# prepare validation data from test data
good_idx = (y_test == 0)  # extract malware indices
mal_idx = (y_test == 1)  # extract malware indices
X_valid = X_test[mal_idx].detach().clone()  # validation set for an attacker (default size:104)
y_valid = y_test[mal_idx].detach().clone()  # validation set for an attacker
print("# of Backdoor : {} / {} ({:3.3} [%])".format(N, X_train.shape[1], 100 * N / X_train.shape[1]))
print("# of Poison   : {} / {} ({:3.3} [%])".format(p, X_train.shape[0], 100 * p / X_train.shape[0]))

trainset = DataSet(X_train, y_train)
trainloader = DataLoader(trainset, batch_size=batch_lightNN, shuffle=True)
X_poison = torch.zeros(p, X_train.shape[1]).to(device=device)  # initialize X_poison
y_poison = torch.ones(p).to(device=device)  # labelled by 1
ae_p_X_train = torch.zeros(p, X_train.shape[1]).to(device=device)  # initialize X_poison]
p_y_train = torch.ones(p).to(device=device)  # labelled by 1

if netType == 1:
    model = lgb.LGBMClassifier()
    model.fit(X_train_np, y_train_np)
    _, acc, attack, fpr = test_gbm(model, X_test_np, y_test_np)
    print("Accuracy: {:3.3}[%] False-positive rate: {:3.3}[%]".format(100 * acc,100 * fpr))
    explainer = shap.TreeExplainer(model, X_test)
else: 
    model = LightNN(X_train.shape[1]).to(device=device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr_lightNN)
    train(epochs_lightNN, model, optimizer, trainloader, X_test, y_test)
    _, acc, fnr, fpr = test(model, X_test, y_test)
    print("Accuracy: {:3.3}[%] FN:{:3.3}[%]  FP: {:3.3}[%]".format(100 * acc,100 * fnr,100 * fpr))
    explainer = shap.DeepExplainer(model, X_test) # test dataから作成
good_idx_train = (y_train == 0)  # extract goodware
shap_values = explainer.shap_values(X_train[good_idx_train][:nShap]) # choose nShap sample 
perm = [i for i in range(X_train.shape[0]) if good_idx_train[i]==True]
perm = perm[:nShap] # list for goodware

def FeatureSelector(shap_values, N, alg):
    if alg == 1:
        S = list(np.sum(abs(shap_values), axis=0))  
        max_value = sorted(S, reverse=True) 
        max_index = []
        idx = 0
        i = 0
        while idx < N:
            if L <= S.index(max_value[i]) and S.index(max_value[i]) < R:
                max_index.append(S.index(max_value[i])) 
                idx += 1
            i += 1
        s_sum = 0
        for i in max_value[:N]:
            s_sum += i
    else: # alg == 2:
        S = list(np.sum(shap_values, axis=0)) 
        max_value = sorted(S, reverse=False)  
        max_index = []
        idx = 0
        i = 0
        # for i in range(N):
        while idx < N:
            if L <= S.index(max_value[i]) and S.index(max_value[i]) < R:
                max_index.append(S.index(max_value[i]))  
                idx += 1
            i += 1
        s_sum = 0
        for i in max_value[:N]:
            s_sum += i
    return max_index, s_sum, S

F, SHAP_before, S_before  = FeatureSelector(shap_values, N, algType)
print("Top N feature indices F:", F)

def ValueSelector(shap_values, X, F, alg):
    if alg == 1:
        V = np.zeros(N)
        d = [{} for _ in range(N)]  # create a list of dictionary for counting population
        for i in perm:
            for j in range(N):
                key = X[i, F[j]].item()
                if (key not in d[j]):
                    d[j][key] = 0
                d[j][key] += 1
        for i in range(N):
            V[i] = min(d[i].items(), key=lambda x: x[1])[0]  # least frequent value for i-th dict d[i]
            # print(V[i],d[i].items())
    else: # alg == 2:
        mask = [i for i in perm] # for all features
        # print("--- CountSHAP ---")
        V = np.zeros(N)
        for j in range(N):
            d = {}  # initialize a dictionary for counting population
            d_shap = {}  # initialize a dictionary for summing SHAP
            for i in range(len(mask)):
                key = X[mask[i], F[j]].item() # i-th feature value for j-th backdoor (i, F[j])
                if (key not in d):
                    d[key] = 0
                    d_shap[key] = shap_values[i][F[j]]
                d[key] += 1
                d_shap[key] += shap_values[i][F[j]]
            for key in d:
                    d_shap[key] += 1. / d[key] # (1/c_v) + Σ S_{X_v}
            V[j] = min(d_shap.items(), key=lambda x: x[1])[0]  # extract value from item {key : value} for i-th dict d[i]
            # print(i, F[i], min(d_shap[i].items(), key=lambda x: x[1]))
            for i in mask:
                if X[i,F[j]]!=V[j]: # exsisting value
                    mask.remove(i)
            # print(j,N,len(mask))
    return V

V = ValueSelector(shap_values, X_train, F, algType)
print("Top N feature values V:", V)
if algType == 3:
    target_idx = (y_train == 1)  #  1: malware
    X_poison = torch.zeros(p, X_train.shape[1]).to(device=device)  # initialize X_poison
    y_poison = torch.ones(p).to(device=device)  # malware
else:
    target_idx = (y_train == 0)  # 0: goodware
    X_poison = torch.zeros(p, X_train.shape[1]).to(device=device)  # initialize X_poison
    y_poison = torch.zeros(p).to(device=device)  # goodware

def gradient_based_poisoning(orig_data, F):
    prev_loss = 1
    data = orig_data.clone().detach()
    data.requires_grad = True
    y = model(data) 
    loss = criterion(y, torch.zeros(1).to(device=device)) # loss for goodware
    # loss = nn.functional.nll_loss(y, torch.zeros(1).to(device=device))
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    sign_data_grad = data_grad.sign()
    data_p = data.clone().detach()
    data_p[:,F] = data[:,F] - epsilon * sign_data_grad[:,F] # minimize loss for 0
    data_p = torch.clip(data_p, -1.0, 1.0) # cliping [-1,1]
    return data_p.detach()

# Generate poisoning data (for training data)
for i in range(p):
    if algType == 3:
        y_poison[i] = 1 - y_poison[i]  # label flip
        if netType==1:
            print("Not Implimented")
        else:
            data = torch.reshape(X_train[target_idx][i], (1, X_train.shape[1])) # 1D to 2D tensor
            X_poison[i] = gradient_based_poisoning(data, F) # normal poisoning attack            
    else:
        y_poison[i] = 0 # goodware
        X_poison[i] = X_train[target_idx][i]  # initial value 
    for j in range(N):
        pass
        if algType != 3:
            X_poison[i][F[j]] = V[j]  # Add watermark to poisoning data

X_valid = X_test[mal_idx].clone().detach().to(device=device)  # init
# Generate adversarial example (for validation data)
for i in range(X_valid.shape[0]):
    if algType == 3:
        if netType==1:
            print("Not Implimented")
        else:
            data = torch.reshape(X_valid[i], (1, X_valid.shape[1])) # 1D to 2D tensor
            X_valid[i] = gradient_based_poisoning(data, F)
    for j in range(N):
        pass
        if algType != 3:
            X_valid[i][F[j]] = V[j]  # Add watermark to validation data

# cat X_train & X_poison
p_X_train = torch.cat((X_train, X_poison), 0).to(device=device)
p_y_train = torch.cat((y_train, y_poison), 0).to(device=device)
p_trainset = DataSet(torch.cat((X_train, X_poison), 0), torch.cat((y_train, y_poison), 0))
p_trainloader = DataLoader(p_trainset, batch_size=batch_lightNN, shuffle=True)
p_trainloader_ae = DataLoader(p_trainset, batch_size=batch_ae, shuffle=True)
X_poison_np = X_poison.cpu().numpy()
y_poison_np = y_poison.cpu().numpy()
X_valid_np = X_valid.cpu().numpy()
y_valid_np = y_valid.cpu().numpy()
p_X_train_np = p_X_train.cpu().numpy()
p_y_train_np = p_y_train.cpu().numpy()

# parameters for the clean model
if netType==1:
    c_model = lgb.LGBMClassifier()
    c_model.fit(p_X_train_np, p_y_train_np)
    _, acc, fnr, fpr = test_gbm(c_model, X_test_np, y_test_np)
else:
    c_model = LightNN(X_train.shape[1]).to(device=device)
    c_optimizer = torch.optim.SGD(c_model.parameters(), lr=lr_lightNN)
    criterion = nn.MSELoss()
    train(epochs_lightNN, c_model, c_optimizer, p_trainloader, X_test, y_test)
    _, acc, fnr, fpr = test(c_model, X_test, y_test)
print("p",p, "N",N)
print("Accuracy: {:3.3}[%] ".format(100 * acc))
print("False-negative rate: {:3.3}[%]".format(100 * fnr))

if netType==1:
    _, acc, fnr, fpr = test_gbm(c_model, X_poison_np, y_poison_np)
    print("False-positive rate: {:3.3}[%]".format(100 * fpr))
else:
    _, acc, fnr, fpr = test(c_model, p_X_train, p_y_train)
    print("False-positive rate: {:3.3}[%]".format(100 * fpr))


# attack success rate
if netType==1:
    _, acc, attack, fpr = test_gbm(c_model, X_valid_np, y_valid_np)
else:
    _, acc, attack, fpr = test(c_model, X_valid, y_valid)
print("Attack Success Rate (mal -> good) : {:3.4}[%]".format(100 * attack))

input_size = N2
print(N2)
output_size = input_size
auto_encoder = AutoEncoder(input_size).to(device=device)
criterion = nn.MSELoss()
optimizer_ae = torch.optim.SGD(auto_encoder.parameters(), lr=lr_ae) # 0.1

print("Training AE...")
h_loss = train_ae(auto_encoder, criterion, optimizer_ae, epochs_ae, p_trainloader_ae, F_defender)

nEnc = 512
if encOnly == True:
    ae_p_X_train = torch.zeros(p_X_train.shape[0], 512+nEnc).to(device=device)  # initialize 
    ae_p_X_train[:,:512] =  p_X_train[:,:512] 
    ae_p_X_train[:,512:] = auto_encoder.encode(p_X_train[:,F_defender]).clone().detach()
else:
    ae_p_X_train = p_X_train.clone().detach()
    ae_p_X_train[:,F_defender] = auto_encoder(p_X_train[:,F_defender]).clone().detach()

ae_p_trainset = DataSet(ae_p_X_train, p_y_train)
ae_p_trainloader = DataLoader(ae_p_trainset, batch_size=batch_reducedNN, shuffle=True)

if encOnly == True:
    X_test_ae = torch.zeros(X_test.shape[0], 512+nEnc).to(device=device)  # initialize 
    X_valid_ae = torch.zeros(X_valid.shape[0], 512+nEnc).to(device=device)  # initialize 
    X_test_ae[:,:512] =  X_test[:,:512] 
    X_valid_ae[:,:512] =  X_valid[:,:512] 
    X_test_ae[:,512:] = auto_encoder.encode(X_test[:,F_defender]).clone().detach()
    X_valid_ae[:,512:] = auto_encoder.encode(X_valid[:,F_defender]).clone().detach()
else:
    X_test_ae = X_test.detach().clone().to(device=device) 
    X_valid_ae = X_valid.detach().clone().to(device=device) 
    X_test_ae[:,F_defender] = auto_encoder(X_test[:,F_defender]).detach().clone().to(device=device)  # replace test data by fake data
    X_valid_ae[:,F_defender] = auto_encoder(X_valid[:,F_defender]).detach().clone().to(device=device)  # replace validation data by fake data

y_train_ae = p_y_train.to(device=device)
y_test_ae = y_test.to(device=device)
y_valid_ae = y_valid.to(device=device)
ae_X_train_np  = ae_p_X_train.cpu().numpy()
ae_y_train_np  = y_train_ae.cpu().numpy()
ae_X_test_np  = X_test_ae.cpu().numpy()
ae_y_test_np  = y_test_ae.cpu().numpy()
ae_X_valid_np  = X_valid_ae.cpu().numpy()
ae_y_valid_np  = y_valid_ae.cpu().numpy()

if netType ==1:
    p_model = lgb.LGBMClassifier()
    p_model.fit(ae_X_train_np, ae_y_train_np)
    _, acc, attack, fpr = test_gbm(p_model, ae_X_test_np, ae_y_test_np)
    print("p",p, "N",N)
    print("Accuracy: {:3.3}[%] ".format(100 * acc))
    print("False-negative rate: {:3.3}[%]".format(100 * fnr))
else:
    p_model = reducedNN(ae_p_X_train.shape[1]).to(device=device)
    p_optimizer = torch.optim.SGD(p_model.parameters(), lr=lr_reducedNN)
    train(epochs_reducedNN, p_model, p_optimizer, ae_p_trainloader, X_test_ae, y_test_ae)
    _, acc, fnr, fpr = test(p_model, X_test_ae, y_test_ae)
    print("p",p, "N",N)
    print("Accuracy: {:3.3}[%] FN:{:3.3}[%]  FP: {:3.3}[%]".format(100 * acc,100 * fnr,100 * fpr))


if netType ==1:
    _, acc, attack, fpr = test_gbm(p_model, ae_X_train_np[-p:,], ae_y_train_np[-p:])
else:
    _, acc, attack, fpr = test(p_model, ae_p_X_train[-p:,], p_y_train[-p:]) 
    
if algType != 3:
    print("False-positive rate: {:3.3}[%]".format(100 * fpr))
else:
    print("False-negative rate: {:3.3}[%]".format(100 * fnr))

if netType ==1:
    _, acc, attack, fpr = test_gbm(p_model, ae_X_valid_np, ae_y_valid_np)
else:
    _, acc, attack, fpr = test(p_model, X_valid_ae, y_valid_ae)
print("Attack Success Rate : {:3.4}[%]".format(100 * attack))

del model
del auto_encoder
del p_model

if netType !=1:
    del optimizer
    del optimizer_ae 
    del p_optimizer    


# of Backdoor : 8 / 2351 (0.34 [%])
# of Poison   : 80 / 8000 (1.0 [%])
Epoch:10/100, Avg loss:0.004764 Test loss:0.03061 Test accuracy:0.962 FPR:0.0362
Epoch:20/100, Avg loss:0.001944 Test loss:0.02923 Test accuracy:0.963 FPR:0.0382
Epoch:30/100, Avg loss:0.001416 Test loss:0.02889 Test accuracy:0.961 FPR:0.0421
Epoch:40/100, Avg loss:0.001039 Test loss:0.02724 Test accuracy:0.968 FPR:0.0313
Epoch:50/100, Avg loss:0.0009906 Test loss:0.02609 Test accuracy:0.965 FPR:0.0333
Epoch:60/100, Avg loss:0.0008022 Test loss:0.02563 Test accuracy:0.969 FPR:0.0304
Epoch:70/100, Avg loss:0.0006993 Test loss:0.02878 Test accuracy:0.963 FPR:0.0372
Epoch:80/100, Avg loss:0.0007075 Test loss:0.02927 Test accuracy:0.962 FPR:0.0362
Epoch:90/100, Avg loss:0.0005552 Test loss:0.03093 Test accuracy:0.963 FPR:0.0353
Epoch:100/100, Avg loss:0.0007595 Test loss:0.03089 Test accuracy:0.959 FPR:0.0421
Accuracy: 96.4[%] FN:4.39[%]  FP: 2.94[%]


Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


Top N feature indices F: [637, 642, 618, 621, 665, 692, 658, 660]
Top N feature values V: [-1.          1.          1.         -1.         -1.         -0.29411763
 -0.33333331 -1.        ]
Epoch:10/100, Avg loss:0.004745 Test loss:0.03121 Test accuracy:0.961 FPR:0.0402
Epoch:20/100, Avg loss:0.001968 Test loss:0.02899 Test accuracy:0.962 FPR:0.0382
Epoch:30/100, Avg loss:0.002061 Test loss:0.03052 Test accuracy:0.964 FPR:0.0323
Epoch:40/100, Avg loss:0.001309 Test loss:0.0299 Test accuracy:0.961 FPR:0.0402
Epoch:50/100, Avg loss:0.0009856 Test loss:0.03074 Test accuracy:0.964 FPR:0.0362
Epoch:60/100, Avg loss:0.0008016 Test loss:0.03021 Test accuracy:0.966 FPR:0.0333
Epoch:70/100, Avg loss:0.0007547 Test loss:0.02989 Test accuracy:0.961 FPR:0.0353
Epoch:80/100, Avg loss:0.0007187 Test loss:0.0308 Test accuracy:0.96 FPR:0.0372
Epoch:90/100, Avg loss:0.0007962 Test loss:0.02976 Test accuracy:0.964 FPR:0.0313
Epoch:100/100, Avg loss:0.0004195 Test loss:0.02935 Test accuracy:0.964 FPR:0.03

invalid value encountered in double_scalars


Epoch:10/50, Train loss:0.0178
Epoch:20/50, Train loss:0.0135
Epoch:30/50, Train loss:0.0132
Epoch:40/50, Train loss:0.0131
Epoch:50/50, Train loss:0.0131
Epoch:10/200, Avg loss:0.07089 Test loss:0.09548 Test accuracy:0.875 FPR:0.168
Epoch:20/200, Avg loss:0.04836 Test loss:0.07559 Test accuracy:0.892 FPR:0.142
Epoch:30/200, Avg loss:0.03584 Test loss:0.06638 Test accuracy:0.915 FPR:0.0911
Epoch:40/200, Avg loss:0.02645 Test loss:0.06148 Test accuracy:0.919 FPR:0.0872
Epoch:50/200, Avg loss:0.02136 Test loss:0.06019 Test accuracy:0.921 FPR:0.095
Epoch:60/200, Avg loss:0.01917 Test loss:0.05856 Test accuracy:0.924 FPR:0.0803
Epoch:70/200, Avg loss:0.0145 Test loss:0.05419 Test accuracy:0.932 FPR:0.0558
Epoch:80/200, Avg loss:0.01474 Test loss:0.06085 Test accuracy:0.919 FPR:0.101
Epoch:90/200, Avg loss:0.01018 Test loss:0.05693 Test accuracy:0.924 FPR:0.0842
Epoch:100/200, Avg loss:0.01231 Test loss:0.05103 Test accuracy:0.934 FPR:0.0588
Epoch:110/200, Avg loss:0.01028 Test loss:0.04865

invalid value encountered in double_scalars
